In [0]:
# --------------------------------------------------------
# 01. Bronze | Ingestão de dados da API pública IBGE SIDRA
#     Tabela: 6579
# --------------------------------------------------------

import requests
from pyspark.sql import functions as F

# 1. Definição da fonte de dados
SIDRA_API_URL = "https://apisidra.ibge.gov.br/values/t/6579/n3/all/p/all/v/all"

# 2. Consumo da API (batch)
response = requests.get(SIDRA_API_URL, timeout=120)
response.raise_for_status()
data = response.json()

# 3. Criação do DataFrame bruto
df_raw = spark.createDataFrame(data)

# 4. Enriquecimento com metadados técnicos
df_bronze = (
    df_raw
    .withColumn("ingestion_ts", F.current_timestamp())
    .withColumn("source", F.lit("IBGE - SIDRA"))
    .withColumn("source_url", F.lit(SIDRA_API_URL))
)

# 5. Remoção explícita da tabela Bronze (camada reprocessável)
spark.sql("DROP TABLE IF EXISTS main.medallion.bronze_sidra_t6579")

# 6. Escrita da camada Bronze (Delta gerenciada)
(
    df_bronze
    .write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("main.medallion.bronze_sidra_t6579")
)

# 7. Evidências de carga
print("Bronze gravado com sucesso: main.medallion.bronze_sidra_t6579")
display(df_bronze.limit(10))
print("Total de linhas:", df_bronze.count())
